In [1]:

# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import os


import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp



import tensorflow.keras as keras
from tensorflow.keras import models, layers

# ======================================================================================================================

scaler = MinMaxScaler()
random.seed(54654)

matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

label = pd.read_csv('train_label.csv')

# ======================================================================================================================
print("process - 1")

activity = pd.read_csv('train_activity.csv')

# activity acc_id로 groupby
# - 평균을 내지 않은 이유 : 평균을 냈을 경우 캐릭터는 많지만
#   한 캐릭터만으로 활동한 사람의 정보가 과소평가 될 가능성이 있음
activity = activity.groupby(['acc_id'], as_index=False).sum()
activity.drop(columns=['day', 'char_id'], inplace=True)
# print(activity.head())


# ======================================================================================================================


combat = pd.read_csv('train_combat.csv')

combat.drop(columns=['day', 'server', 'char_id', 'class'], inplace=True)
combat_a = combat.groupby(['acc_id'], as_index=False).sum()
combat_a.drop(columns='level', inplace=True)

combat_b = combat.groupby(['acc_id'], as_index=False).max()
#  combat.groupby('acc_id', as_index = False).sum().sort_values('acc_id')

# acc_id 기준으로 정리

combat_b = combat_b[['acc_id', 'level']]


##combat_a acc_id별로 pledge_cnt(혈맹전투횟수), random_attacker_cnt, random_defender_cnt
##temp_cnt, same_pledge_cnt, etc_cnt 등등 여러 전투횟수의 총합을 가지고 있다
##combat_b는 acc_id별 가장 높은 레벨을 보여준다
combat = pd.merge(combat_b, combat_a, how='inner', on='acc_id')

# ======================================================================================================================


payment = pd.read_csv('train_payment.csv')

payment = payment.groupby('acc_id', as_index=False).sum()
payment.drop(columns='day', inplace=True)

payment.rename(columns={'amount_spent': 'amount_spent_pay'}, inplace=True)

# ======================================================================================================================


trade = pd.read_csv('train_trade.csv')

# 거래에 참여한 횟수를 기준으로

# 판매자로서 활동한 acc_id
trade_a = trade.groupby('source_acc_id', as_index=False).count()
trade_a = trade_a[['source_acc_id', 'day']]

# 구매자로서 활동한 acc_id
trade_b = trade.groupby('target_acc_id', as_index=False).count()
trade_b = trade_b[['target_acc_id', 'day']]

x = trade_a['day'].sum() - trade_b['day'].sum()
# print(x) # 0

trade_a.rename(columns={'source_acc_id': 'acc_id',
                        'day': 'sell_item_cnt'}, inplace=True)
trade_b.rename(columns={'target_acc_id': 'acc_id',
                        'day': 'buy_item_cnt'}, inplace=True)

trade = pd.merge(trade_a, trade_b, how='outer', on='acc_id')

# 실제 데이터 검색
# trade[trade['source_acc_id'] == 6].count()

# 데이터 검증
# trade_a[trade_a['source_acc_id'] == 6]


# ======================================================================================================================


pledge = pd.read_csv('train_pledge.csv')

ple_1 = pledge.groupby(['server', 'pledge_id', 'day'], as_index=False).mean()
ple_1.drop(columns=['acc_id', 'char_id', 'day'], inplace=True)
ple_1 = ple_1.groupby(['server', 'pledge_id'], as_index=False).sum()

ple_a = pledge.groupby(['acc_id', 'char_id', 'server', 'pledge_id', 'day'], as_index=False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'server', 'pledge_id'], as_index=False).sum()
ple_a = ple_a[['acc_id', 'char_id', 'server', 'pledge_id']]

pledge = pd.merge(ple_a, ple_1, how='outer', on=['server', 'pledge_id'])
pledge.drop(columns=['char_id', 'server', 'pledge_id'], inplace=True)
pledge_total = pledge.groupby(['acc_id'], as_index=False).mean()

pledge_total.rename(columns={'random_attacker_cnt': 'random_attacker_cnt_plg',
                             'random_defender_cnt': 'random_defender_cnt_plg',
                             'same_pledge_cnt': 'same_pledge_cnt_plg',
                             'temp_cnt': 'temp_cnt_plg',
                             'etc_cnt': 'etc_cnt_plg'}, inplace=True)

# ple_1[(ple_1['pledge_id'] == 25467) & (ple_1['server'] == 'aq')]
# ple_a[(ple_a['pledge_id'] == 25467) & (ple_a['server'] == 'aq')]


# ======================================================================================================================
print("process - 2")

# label + activity
label_a = pd.merge(label, activity, how='outer', on='acc_id')

# (label + activity) + combat
label_b = pd.merge(label_a, combat, how='outer', on='acc_id')

# (label + activity + combat) + payment
label_c = pd.merge(label_b, payment, how='outer', on='acc_id')

# (label + activity + combat + payment) + trade
label_d = pd.merge(label_c, trade, how='outer', on='acc_id')
label_d = label_d[label_d['survival_time'] >= 1]

# (label + activity + combat + payment + trade) + pledge_total
label_z = pd.merge(label_d, pledge_total, how='outer', on='acc_id')

data = label_z.fillna(0)
print(data.columns)

data[['playtime', 'npc_kill',
      'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
      'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
      'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
      'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
      'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
      'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
      'random_attacker_cnt_plg', 'random_defender_cnt_plg',
      'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
      'combat_play_time', 'non_combat_play_time']] = \
    scaler.fit_transform(data[['playtime', 'npc_kill',
                               'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
                               'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
                               'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
                               'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
                               'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
                               'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
                               'random_attacker_cnt_plg', 'random_defender_cnt_plg',
                               'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
                               'combat_play_time', 'non_combat_play_time']])

data = data.fillna(0)
print(data.columns)

# ======================================================================================================================

data.loc[data['amount_spent_pay'] == 0, "cash"] = 0
data.loc[data['amount_spent_pay'] > 0, "cash"] = 1

# ======================================================================================================================


# 무과금

data_1 = data[data["cash"] == 0]
print(data_1.count())
data_1_corr = data_1.corr()

data_1.loc[data_1['survival_time'] < 64, "survived"] = 0
data_1.loc[data_1['survival_time'] == 64, "survived"] = 1

data_1_2 = data_1.corr()

# ======================================================================================================================

# 무과금 + 이탈
print("process - 3")
data_2 = data_1[data_1["survived"] == 0]

data_2_1 = data_2.corr()
"""

# 서로 반비례관계에 있을 시 제외

survival_time :
    playtime # 0.19
        npc_kill                # 0.5
        party_exp               # 0.17
        death                   # 0.2
        revive                  # 0.15
        private_shop            # 0.33
        level                   # 0.13
        random_defender_cnt     # 0.37
        etc_cnt                 # 0.15
        num_opponent            # 0.17
        sell_item_cnt           # 0.15
        random_defender_cnt_plg # 0.15
        combat_play_time        # 0.29
        non_combat_play_time    # 0.33
    npc_kill # 0.15
        rich_monster            # 0.12
        death                   # 0.11
        level                   # 0.40
        random_defender_cnt     # 0.26
        etc_cnt                 # 0.24
        num_opponent            # 0.23
    rich_monster
        solo_exp                # 0.47
        quest_exp               # 0.30
        death                   # 0.22
        revive                  # 0.23
        level                   # 0.17
        num_opponent            # 0.24
        play_char_cnt           # 0.14
        pledge_combat_cnt       # 0.14
        random_defender_cnt_plg # 0.18
    level
        combat_play_time        # 0.31
    num_opponent
        pledge_cnt              # 0.41

"""
# print(data_2.columns)
x_e = data_1[['acc_id', 'playtime', 'npc_kill',
              'solo_exp', 'quest_exp',
              'private_shop', 'level', 'pledge_cnt', 'random_defender_cnt', 'etc_cnt',
              'num_opponent', 'sell_item_cnt',
              'combat_play_time', 'non_combat_play_time']]
y_e = data_1[['acc_id', 'survival_time']]


xy_e = pd.merge(x_e, y_e, how='outer', on='acc_id')


xy_e.to_csv('merge.csv',index=False)


def merge_data(income_path):
    csv_path = os.path.join(income_path,"merge.csv")
    return pd.read_csv(csv_path,sep=',',engine='python')


result = merge_data(".")


LABEL_COLUMN = 'survival_time'
LABLES = [0,1]


def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(file_path,
                                                    batch_size=32, # Artificially small to make examples easier to show.
                                                    label_name=LABEL_COLUMN,
                                                    na_value="?",
                                                    num_epochs=1,
                                                    num_parallel_reads=1,
                                        
                            
                                                    **kwargs)
    return dataset



PATH = './merge.csv'
raw_train_data = get_dataset(PATH)

conti_var = result.columns[result.dtypes != 'object'] # 숫자로 된 데이터만 뽑아온다
list(conti_var)


process - 1
process - 2
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', '

C:\Users\hyun\Anaconda3\envs\ilhyun\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\hyun\Anaconda3\envs\ilhyun\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


['acc_id',
 'playtime',
 'npc_kill',
 'solo_exp',
 'quest_exp',
 'private_shop',
 'level',
 'pledge_cnt',
 'random_defender_cnt',
 'etc_cnt',
 'num_opponent',
 'sell_item_cnt',
 'combat_play_time',
 'non_combat_play_time',
 'survival_time']

In [2]:
va = list(conti_var)
va
del va[14]
va


['acc_id',
 'playtime',
 'npc_kill',
 'solo_exp',
 'quest_exp',
 'private_shop',
 'level',
 'pledge_cnt',
 'random_defender_cnt',
 'etc_cnt',
 'num_opponent',
 'sell_item_cnt',
 'combat_play_time',
 'non_combat_play_time']

In [3]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_freatures = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        matches = tf.greater_equal(10.0, labels)
        onehot = tf.cast(matches, tf.float32)
        labels = onehot
        
        return features, labels

In [4]:
NUMERIC_FEATURES = va

In [5]:
packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [6]:
example_batch, labels_batch = next(iter(packed_train_data))
desc = result[NUMERIC_FEATURES].describe()

In [7]:
desc

,acc_id,playtime,npc_kill,solo_exp,quest_exp,private_shop,level,pledge_cnt,random_defender_cnt,etc_cnt,num_opponent,sell_item_cnt,combat_play_time,non_combat_play_time
count,16274.000000,16274.000000,16274.000000,16274.000000,1.627400e+04,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000
mean,64998.933083,0.563972,0.044365,0.017959,1.447120e-03,0.244198,0.652249,0.001200,0.017934,0.003843,0.006023,0.010857,0.057017,0.025539
std,37549.375996,0.405614,0.065457,0.044556,5.610076e-03,0.395863,0.382641,0.014804,0.055584,0.015981,0.018199,0.037883,0.093867,0.091498
min,2.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32664.750000,0.087235,0.000000,0.000000,0.000000e+00,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,64660.000000,0.742085,0.006186,0.000656,8.497309e-07,0.000000,0.882353,0.000000,0.000000,0.000000,0.000367,0.000650,0.019915,0.000000
75%,97531.750000,0.963161,0.072583,0.008131,9.027428e-05,0.724334,0.941176,0.000000,0.011407,0.001363,0.004581,0.005631,0.079977,0.003840
max,130468.000000,1.000000,0.468380,0.479215,1.531372e-01,1.000000,1.000000,0.809812,1.000000,0.486318,0.812534,1.000000,1.000000,1.000000


In [8]:
MEAN = np.array(desc.T['mean'])

In [9]:
STD = np.array(desc.T['std'])


In [10]:
def normalize_numeric_data(data,mean,std):
    return (data-mean)/std

In [11]:
import functools

In [12]:
normalizer = functools.partial(normalize_numeric_data,mean=MEAN,std=STD)
numeric_column = tf.feature_column.numeric_column('numeric',normalizer_fn=normalizer,shape=[len(NUMERIC_FEATURES)])

In [13]:
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(14,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000023B076D1B88>, mean=array([6.49989331e+04, 5.63971511e-01, 4.43651184e-02, 1.79594110e-02,
       1.44711954e-03, 2.44197668e-01, 6.52249348e-01, 1.20007182e-03,
       1.79343196e-02, 3.84265088e-03, 6.02299016e-03, 1.08569462e-02,
       5.70173481e-02, 2.55394173e-02]), std=array([3.75493760e+04, 4.05614069e-01, 6.54574672e-02, 4.45556379e-02,
       5.61007627e-03, 3.95862946e-01, 3.82641193e-01, 1.48041768e-02,
       5.55844748e-02, 1.59805615e-02, 1.81989521e-02, 3.78829822e-02,
       9.38674781e-02, 9.14978125e-02])))

In [14]:
example_batch['numeric']
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()
categorical = list(result.columns[result.dtypes == 'object'])
categorical
def get_keys(data,feature):
    return list(result[feature].value_counts().keys())
def get_categories(data,c_list):
    category={}
    for i in c_list[:-1]:
        category[i] = get_keys(data,i)
    return category
CATEGORIES = get_categories(result, categorical)
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, 
                                                                        vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    categorical_columns[0]

In [15]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numeric_columns)

In [16]:
print(preprocessing_layer(example_batch).numpy()[0])

[-1.6906523   1.0610424  -0.67777014 -0.4030783  -0.25795007  1.7828506
 -1.7045978  -0.08106305 -0.3226498  -0.2404578  -0.33095258  0.9197725
 -0.6074239  -0.27912596]


In [17]:
train_data = packed_train_data.take(900).shuffle(100)

In [18]:
test_data = packed_train_data.skip(500)

In [19]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [20]:
model.fit(train_data, epochs=50)

Epoch 1/50
509/509 [==============================] - 3s 7ms/step - loss: 0.4095 - accuracy: 0.8302
Epoch 2/50
509/509 [==============================] - 1s 3ms/step - loss: 0.3910 - accuracy: 0.8372
Epoch 3/50
509/509 [==============================] - 1s 2ms/step - loss: 0.3806 - accuracy: 0.8440
Epoch 4/50
509/509 [==============================] - 1s 3ms/step - loss: 0.3731 - accuracy: 0.8509
Epoch 5/50
509/509 [==============================] - 1s 3ms/step - loss: 0.3652 - accuracy: 0.8515
Epoch 6/50
509/509 [==============================] - 2s 4ms/step - loss: 0.3581 - accuracy: 0.8569
Epoch 7/50
509/509 [==============================] - 1s 3ms/step - loss: 0.3518 - accuracy: 0.8614
Epoch 8/50
509/509 [==============================] - 2s 3ms/step - loss: 0.3465 - accuracy: 0.8647
Epoch 9/50
509/509 [==============================] - 2s 4ms/step - loss: 0.3372 - accuracy: 0.8688
Epoch 10/50
509/509 [==============================] - 2s 3ms/step - loss: 0.3327 - accuracy: 0.8735

In [21]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

9/9 [==============================] - 1s 62ms/step - loss: 0.1920 - accuracy: 0.9270


Test Loss 0.19197576410240597, Test Accuracy 0.9270073175430298


In [22]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:32], list(test_data)[0][1]):   # outcome of one batch
    print("Predicted serviaval_time: {:.2%}".format(prediction[0]),
        " | Actual servival_time: ",
        (">=10" if bool(survived) else "<10"))

Predicted serviaval_time: 2.12%  | Actual servival_time:  <10
Predicted serviaval_time: 73.77%  | Actual servival_time:  >=10
Predicted serviaval_time: 0.01%  | Actual servival_time:  <10
Predicted serviaval_time: 3.49%  | Actual servival_time:  >=10
Predicted serviaval_time: 18.06%  | Actual servival_time:  <10
Predicted serviaval_time: 7.00%  | Actual servival_time:  >=10
Predicted serviaval_time: 4.56%  | Actual servival_time:  <10
Predicted serviaval_time: 59.69%  | Actual servival_time:  >=10
Predicted serviaval_time: 18.40%  | Actual servival_time:  <10
Predicted serviaval_time: 1.99%  | Actual servival_time:  <10
Predicted serviaval_time: 2.77%  | Actual servival_time:  <10
Predicted serviaval_time: 0.00%  | Actual servival_time:  <10
Predicted serviaval_time: 99.81%  | Actual servival_time:  <10
Predicted serviaval_time: 3.96%  | Actual servival_time:  <10
Predicted serviaval_time: 0.19%  | Actual servival_time:  <10
Predicted serviaval_time: 85.07%  | Actual servival_time:  <1

In [24]:
####################################
################생존시간 구하는 코드 2번째 모델링
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

x = x_e.set_index('acc_id')
y = y_e.set_index('acc_id')

print(y.shape)
print(x.shape)
print(y.columns)
print(y)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=8431)

print(type(X_train))
print(type(Y_train))
print(type(X_test))
print(type(Y_test))





# Build Graph
nb_classes = 64
X = tf.placeholder(tf.float32, shape=[None, 13])
Y = tf.placeholder(tf.int32, shape=[None, 1])

# Y 값을 one_hot encoding으로 변환, Y값은 반드시 int형으로 입력
Y_one_hot = tf.one_hot(Y, nb_classes)                # [None, 1, 7]
print(Y_one_hot)                                     # shape=(?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])  # numpy에서 행을 자동으로 크기에 맞게 다시 조절한다 "-1"
print(Y_one_hot)                                     # [None, 1]


W1 = tf.Variable(tf.random_normal([13, 13]), name='weight')
b1 = tf.Variable(tf.random_normal([13]), name='bias')
logits = tf.matmul(X,W1) + b1

W2 = tf.Variable(tf.random_normal([13, 13]), name='weight')
b2 = tf.Variable(tf.random_normal([13]), name='bias')
logits2 = tf.matmul(logits, W2) + b2


W3 = tf.Variable(tf.random_normal([13, nb_classes]), name='weight')
b3 = tf.Variable(tf.random_normal([nb_classes]), name='bias')
logits3 = tf.matmul(logits,W3) +b3



hypothesis = tf.nn.softmax(logits3)


# cost function
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits =hypothesis,
                                                labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)


# 경사하강법
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습단계(start learning)

for step in range(100001):
    cost_val, W1_val, b1_val,W2_val, b2_val,W3_val, b3_val, _ = \
        sess.run([cost, W1, b1,W2, b2,W3, b3, optimizer],
                 feed_dict = {X : X_train, Y : Y_train})
    if step % 10000 == 0:
        print('\nstep:', step,
              '\ncost_val:', cost_val,)


# 검증단계(test, 정확도 측정)
# Accuracy Computation(정확도 계산)
predict = tf.argmax(hypothesis, 1) # 1:행단위 # 예측값 행에서 가장 큰 값, 확률값을 구한다.
# predict = tf.argmax(hypothesis,0) # 0:열단위

correct_predict = tf.equal(predict, tf.argmax(Y_one_hot, 1))  # predict와 y 값을 비교한다.
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32)) # 숫자로 바꾼 뒤 평균을 낸다.

h, p, a = sess.run([hypothesis, predict, accuracy],feed_dict={X:X_test, Y:Y_test})

print('Hypothesis:' , h,
      '\nPredict:',p,
      '\nAccuracy:', a)

Instructions for updating:
non-resource variables are not supported in the long term
(16274, 1)
(16274, 13)
Index(['survival_time'], dtype='object')
        survival_time
acc_id               
91444            64.0
82976            33.0
89354             1.0
86978            52.0
71096            64.0
...               ...
14925             1.0
53989            51.0
64094            64.0
83195            64.0
99716            39.0

[16274 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Tensor("one_hot:0", shape=(?, 1, 64), dtype=float32)
Tensor("Reshape:0", shape=(?, 64), dtype=float32)

step: 0 
cost_val: 1.94809

step: 10000 
cost_val: 1.8566103

step: 20000 
cost_val: 1.8563054

step: 30000 
cost_val: 1.8562381

step: 40000 
cost_val: 1.8560469

step: 50000 
cost_val: 1.8560449

step: 60000 
cost_val: 1.8560442

step: 70000 
cost_val: 1.8559299

step: 80000 
cost

In [25]:
#1~64의 생존시간을 랜덤포레스트 예측해보기
# 세 번째 모델링
# data_cash_yes.columns

data_cash_yes = data[['combat_play_time','death','enchant_count','etc_cnt',
           'etc_cnt_plg','exp_recovery','level','non_combat_play_time',
           'num_opponent','party_exp','play_char_cnt','pledge_combat_cnt',
           'private_shop','quest_exp','random_attacker_cnt','random_attacker_cnt_plg',
           'random_defender_cnt','random_defender_cnt_plg','same_pledge_cnt_plg',
           'sell_item_cnt','temp_cnt','temp_cnt_plg','survival_time']]

# [data_cash['survival_predict']==0]
y_data = [[i] for i in data_cash_yes['survival_time'].tolist() ]
y= y_data 

data_cash_yes.drop(['survival_time'], axis=1, inplace=True)

x_data = [ data_cash_yes.iloc[i,:].tolist() for i in range(len(data_cash_yes.index.values))]
X = x_data

from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

# multi_variable_linear_regression.py
# 4

C:\Users\hyun\Anaconda3\envs\ilhyun\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
# 학습 진행
rfr = RandomForestClassifier(n_estimators=400, max_depth = 1,random_state=0)
rfr.fit(x_train, y_train)

gbrt = GradientBoostingClassifier(n_estimators=400, max_depth=1, learning_rate=0.01, random_state=0)
gbrt.fit(x_train, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_test, y_test)))
    
    
# 예측 
rfr_y_pred = rfr.predict(X_test)
print(rfr_y_pred)
gbc_y_pred = gbrt.predict(X_test)
print(gbc_y_pred)
print(len(list(y_test)))

# 정확도 확인
print('RandomForestClassifier 정확도 :', metrics.accuracy_score(y_test, rfr_y_pred))
print('GradientBoostingClassifier 정확도 :', metrics.accuracy_score(y_test, gbc_y_pred))

C:\Users\hyun\Anaconda3\envs\ilhyun\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\hyun\Anaconda3\envs\ilhyun\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 세트 정확도: 0.562
테스트 세트 정확도: 0.564
[64. 64. 64. ... 64. 64. 64.]
[64. 64. 64. ... 64. 64. 64.]
13334
RandomForestClassifier 정확도 : 0.5506224688765562
GradientBoostingClassifier 정확도 : 0.563896805159742


In [41]:
## train_pledge를 가지고 모델링을 해봤는데 정확도가 너무 안나와서 여기서부터는 프로젝트 계획서에 작성하지는
## 않았습니다.
##
##
##
##
##





##pledge를 갖고 modeling 한 것

activity = pd.read_csv('train_activity.csv')
label = pd.read_csv('train_label.csv')
pledge = pd.read_csv('train_pledge.csv')

In [29]:
# Pledge ID & Day 로 groupby.mean

ple_1 = pledge.groupby(['pledge_id', 'day'], as_index = False).mean()
ple_1.drop(columns = ['acc_id', 'char_id'], inplace = True)

In [30]:
# pledge ID 로 groupby.sum

ple_2 = ple_1.groupby(['pledge_id'], as_index = False).sum()

In [31]:
ple_2_pcc = ple_2
print(ple_2_pcc.quantile([.25, .5, .75], axis = 0))
q25, q50, q75 = ple_2_pcc["play_char_cnt"].quantile([.25, .5, .75])
print(q25, q50, q75)

      pledge_id    day  play_char_cnt  combat_char_cnt  pledge_combat_cnt  \
0.25    9820.25   28.0       0.072179         0.000000                0.0   
0.50   19792.00   92.0       0.324804         0.000000                0.0   
0.75   29637.50  352.0       1.660109         0.252625                0.0   

      random_attacker_cnt  random_defender_cnt  same_pledge_cnt  temp_cnt  \
0.25                  0.0             0.000000              0.0  0.000000   
0.50                  0.0             0.000000              0.0  0.000000   
0.75                  0.0             0.071531              0.0  0.296649   

       etc_cnt  combat_play_time  non_combat_play_time  
0.25  0.000000          0.000000              0.000347  
0.50  0.000000          0.000000              0.002085  
0.75  0.063704          0.412619              0.041612  
0.07217863870100892 0.32480387415454015 1.6601086901232052


In [32]:
ple_2_pcc.loc[ple_2_pcc["play_char_cnt"] < 0.072179, "play_char_cnt_grade"] = 1
ple_2_pcc.loc[(ple_2_pcc["play_char_cnt"] >= 0.072179) & (ple_2_pcc["play_char_cnt"] < 0.324804), "play_char_cnt_grade"] = 2
ple_2_pcc.loc[(ple_2_pcc["play_char_cnt"] >= 0.324804) & (ple_2_pcc["play_char_cnt"] < 1.660109), "play_char_cnt_grade"] = 3
ple_2_pcc.loc[ple_2_pcc["play_char_cnt"] >= 1.660109, "play_char_cnt_grade"] = 4

In [33]:
# ple_2_pcc 에서 'pledge_id'와 'play_char_cnt_grade'를 추출
print(ple_2_pcc.columns)
new_ple = ple_2_pcc[['pledge_id', 'play_char_cnt_grade']]


ple_a = pledge.groupby(['acc_id', 'char_id', 'pledge_id', 'day'], as_index = False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'pledge_id'], as_index = False).sum()
ple_a.drop(columns = ['day'], inplace = True)
pled = pd.merge(ple_a, new_ple, how = 'inner', on = 'pledge_id').sort_values('acc_id')
pled = pled.groupby(['acc_id'], as_index = False).mean()
pled.drop(columns = ['char_id', 'pledge_id'], inplace = True)



Index(['pledge_id', 'day', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt', 'random_defender_cnt',
       'same_pledge_cnt', 'temp_cnt', 'etc_cnt', 'combat_play_time',
       'non_combat_play_time', 'play_char_cnt_grade'],
      dtype='object')


In [34]:
# 각각의 캐릭터들이 혈맹에서 어떤 활동을 어느 정도 했는지 예상할 수 없기 때문에, 각 캐릭터들이 속한 혈맹의 활동량으로만
# 측정하도록 한다.


# label 과 pledge 를 outer로 결합

label_ple = pd.merge(label, pled, how = 'outer', on = 'acc_id')

# NaN == 0, Not NaN == 1
N_label = label_ple
N_label = N_label.fillna(0)

In [35]:
N_label.loc[N_label["play_char_cnt"] != 0, "NaN_or_Not"] = 1
N_label.loc[N_label["play_char_cnt"] == 0, "NaN_or_Not"] = 0


N_label.loc[N_label["survival_time"] != 64, "sur"] = 0
N_label.loc[N_label["survival_time"] == 64, "sur"] = 1

N_label = N_label.set_index('acc_id')

In [36]:
print(N_label.columns)
X_train, X_test, Y_train, Y_test = train_test_split(N_label[['play_char_cnt', 'non_combat_play_time', 'combat_play_time']], N_label['survival_time'],
                                                    test_size = 0.3, random_state= 123)

print(X_train.shape)
print(X_test.shape)


Index(['survival_time', 'amount_spent', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt', 'random_defender_cnt',
       'same_pledge_cnt', 'temp_cnt', 'etc_cnt', 'combat_play_time',
       'non_combat_play_time', 'play_char_cnt_grade', 'NaN_or_Not', 'sur'],
      dtype='object')
(28000, 3)
(12000, 3)


In [37]:
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)


In [38]:
Y_train = Y_train.reshape(-1, 1)
Y_test = Y_test.reshape(-1, 1)


In [39]:
nb_classes = 64

X = tf.placeholder(tf.float32, shape = [None, 3])
Y = tf.placeholder(tf.int32, shape = [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print('여기-3')
W = tf.Variable(tf.random_normal([3, nb_classes], name = 'weight'))
b = tf.Variable(tf.random_normal([nb_classes], name = 'bias'))
print('여기-4')
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
print('여기-5')
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits,
                                                    labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)
print('여기-6')
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cost)
print('여기-7')
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('여기-8')
for step in range(60000):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, optimizer],
                 feed_dict = {X : X_train, Y : Y_train})
    if step % 5000 == 0:
        print('step:', step,
              'cost_val:', cost_val,
              'W_val', W_val,
              'b_val', b_val)

predict = tf.argmax(hypothesis, 1)
correct_predict = tf.equal(predict, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))

h, p, a = sess.run([hypothesis, predict, accuracy],feed_dict={X:X_test, Y:Y_test})

print('Hypothesis:' , h,
      '\nPredict:', p,
      '\nAccuracy:', a)

여기-3
여기-4
여기-5
여기-6
여기-7
여기-8
step: 0 cost_val: 22.173557 W_val [[ 1.2702568e+00  1.9452643e-01  5.3396512e-02 -7.5255638e-01
  -1.1435730e+00 -5.5391288e-01  1.1975935e+00  4.4825557e-01
  -1.2788075e-01 -1.0640612e-01  1.8178449e+00  2.0513260e+00
   1.0496849e+00  3.1683573e-01  1.3236905e+00 -7.9678887e-01
  -9.8198646e-01  5.4666483e-01  1.2546519e+00 -1.3347064e+00
   9.5243591e-01 -2.1062994e+00 -1.5598528e+00 -1.8151077e+00
  -1.0854695e+00 -1.0504172e+00  4.4291952e-01  8.1002884e-02
   1.7460839e-01  2.2399151e-01  2.3895919e-01  1.7217302e-01
   5.7133341e-01 -3.3945445e-02  1.0833210e+00 -4.3296495e-01
  -2.0710568e-01 -8.9601362e-01  8.7496120e-01 -1.6065359e+00
  -7.2591114e-01  8.8474655e-01 -9.5021141e-01 -9.0743333e-01
   8.5647315e-01 -5.1977396e-01 -5.3210008e-01 -4.0484518e-01
   9.2981541e-01 -6.5180492e-01 -1.4210004e+00 -8.3364320e-01
   2.5594714e+00  1.1987996e+00 -4.2222881e-01  6.0520716e-02
   1.2035158e+00  8.7466043e-01 -1.0664297e+00  4.3520060e-01
  -1.3

step: 15000 cost_val: 1.9845144 W_val [[-145.22902 -145.04105 -144.98672 -145.00243 -145.24199 -145.09476
  -144.97237 -145.21216 -145.27164 -145.05893 -145.01428 -144.86536
  -145.05695 -145.0908  -144.98228 -145.49004 -145.09995 -145.07927
  -145.01326 -145.10075 -145.06178 -145.08282 -145.0771  -145.14374
  -145.17662 -145.18478 -145.2696  -145.09097 -145.0254  -145.24673
  -145.12732 -145.00389 -145.08296 -145.0667  -144.9356  -145.2593
  -145.10938 -145.26424 -145.01106 -145.12328 -145.38106 -145.01222
  -144.99625 -145.19005 -145.15735 -145.06984 -145.10304 -145.05891
  -145.10571 -145.10826 -145.09004 -145.1511  -144.39925 -145.03078
  -145.02663 -145.01768 -145.02393 -145.24889 -145.13869 -144.91968
  -145.4859  -145.0997  -145.12494 -145.09032]
 [-143.66394 -141.19061 -143.13774 -141.90111 -140.19017 -140.43646
  -140.48601 -140.14143 -140.25883 -140.71179 -140.4207  -140.4735
  -140.42886 -140.32224 -140.45602 -140.53996 -140.42346 -140.89851
  -140.47285 -140.31065 -140.3631

step: 30000 cost_val: 1.9856563 W_val [[-295.21255 -295.02536 -294.97107 -294.98666 -295.22665 -295.07922
  -294.9566  -295.19675 -295.25632 -295.04333 -294.99866 -294.84915
  -295.04132 -295.0753  -294.9665  -295.47568 -295.08444 -295.06372
  -294.99756 -295.0853  -295.04626 -295.06732 -295.06158 -295.1283
  -295.16116 -295.16946 -295.2543  -295.0755  -295.00986 -295.23138
  -295.1118  -294.98825 -295.0674  -295.05115 -294.91983 -295.24396
  -295.0939  -295.249   -294.99542 -295.10782 -295.36594 -294.99652
  -294.9806  -295.17468 -295.14188 -295.05426 -295.08755 -295.04337
  -295.09015 -295.09283 -295.07452 -295.13562 -294.37985 -295.0151
  -295.01105 -295.00204 -295.00824 -295.23355 -295.12323 -294.90378
  -295.4711  -295.0841  -295.1095  -295.0748 ]
 [-293.63037 -291.1818  -293.12732 -291.89157 -290.1815  -290.42764
  -290.4773  -290.1327  -290.25015 -290.70288 -290.412   -290.46555
  -290.42014 -290.31357 -290.4474  -290.53107 -290.41476 -290.88947
  -290.46414 -290.30188 -290.3544

step: 45000 cost_val: 1.9852985 W_val [[-445.35904 -445.17163 -445.11734 -445.13287 -445.3727  -445.2254
  -445.10297 -445.34286 -445.40244 -445.18958 -445.14496 -444.99557
  -445.1876  -445.22147 -445.11285 -445.621   -445.23065 -445.20996
  -445.1439  -445.23145 -445.19244 -445.21347 -445.20776 -445.2744
  -445.30728 -445.31555 -445.40033 -445.22165 -445.1561  -445.3775
  -445.25797 -445.13452 -445.21362 -445.1974  -445.06616 -445.39
  -445.2401  -445.39505 -445.14172 -445.25397 -445.51184 -445.14282
  -445.1269  -445.32083 -445.28806 -445.2005  -445.23373 -445.1896
  -445.2364  -445.23895 -445.2207  -445.28177 -444.52634 -445.16138
  -445.1573  -445.14835 -445.15454 -445.37964 -445.26938 -445.05017
  -445.61682 -445.23035 -445.25565 -445.22098]
 [-443.76477 -441.32727 -443.27264 -442.0375  -440.32635 -440.57285
  -440.62234 -440.27777 -440.395   -440.84796 -440.55685 -440.61023
  -440.56503 -440.45844 -440.5923  -440.67624 -440.55972 -441.0346
  -440.60916 -440.44696 -440.4995  -440

Hypothesis: [[5.78391848e-07 4.77757566e-02 1.78786665e-02 ... 1.30867008e-02
  5.50888432e-03 5.90728503e-03]
 [1.44547594e-05 4.48164456e-02 1.53451245e-02 ... 1.24298949e-02
  5.07389288e-03 5.31999115e-03]
 [3.62417800e-03 5.59867658e-02 2.00996269e-02 ... 1.26457652e-02
  4.42771614e-03 5.09564765e-03]
 ...
 [7.16407669e-07 6.14132509e-02 2.76541021e-02 ... 1.35435015e-02
  5.16317319e-03 6.26763469e-03]
 [7.80121638e-07 4.76769060e-02 1.77201256e-02 ... 1.30468747e-02
  5.47337951e-03 5.86348400e-03]
 [3.31788996e-23 1.25590274e-02 2.24765148e-02 ... 1.51172630e-03
  6.47650508e-04 1.31855102e-03]] 
Predict: [57 57 57 ... 57 57 52] 
Accuracy: 0.010833333
